In [ ]:
import numpy as np

In [ ]:
class MSE:
    
    def __init__(self, rate=0.01, cycles=1000):
        self.rate = rate
        self.cycles = cycles
        
    def run(self, points, ids):
        self.theta = np.zeros(np.size(points[0]))
        self.bias = 0
        
        for i in range(self.cycles):
            for point, truth in zip(points, ids):
        
        return self
        
    def predict(self, ds):
        return np.dot(ds, self.theta)

In [ ]:
mse = MSE()

In [ ]:
data = np.load('Data1/data.npz')
x = data['x']
y = np.where(data['y'] > 0, -1, 1)
print(x)
print(y)

In [ ]:
out = mse.run()

In [ ]:
print(out.theta)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

In [ ]:
plot_decision_regions(x, y, clf=mse)